# Random Forest 1

En este primer notebook hacemos un solo random forest para practicar la implementación y ver los parámetros

In [ ]:
# paquetes
import pandas as pd # para maenejar dataframes
from sklearn.model_selection import train_test_split # para dividir la base de datos en train/test
from sklearn.preprocessing import LabelEncoder # para codificar las categorías
from sklearn.ensemble import RandomForestClassifier # para implementar random forest
from sklearn.metrics import confusion_matrix, classification_report # para evaluar las predicciones
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# importamos los datos
df_import = pd.read_csv('df_datos.csv', index_col='Unnamed: 0')

# separamos las variables predictivas (X) de la categoría (y)
df_X = df_import.drop(columns = ['id_ecg','categoria','patient_id'])
ps_y = df_import['categoria'] # ps = pandas series

In [ ]:
# label encoder
# vamos a codificar en números las categorías del dataframe
# aquí tenemos dos funciones unidas: fit y transform.
# - fit: crea el mapeo de las categorías, pero no lo aplica 
# - transform: aplica el mapeo creado en fit en los datos
label_encoder = LabelEncoder()
ps_y_encoded = label_encoder.fit_transform(ps_y)

In [ ]:
# train_test_split
# vamos a dividir los datos para el grupo de entrenamiento y el de prueba.
X_train, X_test, y_train, y_test = train_test_split(df_X, # variables predictivas
                                                    ps_y_encoded, # variable objetivo (ya codificada)
                                                    test_size= 0.8, # proporción del test size
                                                    random_state=10) # fijamos random seed para reproducibilidad

In [ ]:
# entrenamiento del random forest
# establecemos los parámetros del modelo
modelo_abstracto = RandomForestClassifier(n_estimators=20, # número de árboles que se van a generar
                                criterion='gini', # criterio de decisión para la construcción de cada árbol
                                max_depth=None, # máximo número de nodos hacia abajo que puede haber. None = infinito
                                min_samples_split=10, # si más de n eventos llegan a una hoja, entonces se vuelve en nodo y se subdivide en dos hojas.
                                max_features=6, # cuántas columnas se van a considerar en la construcción de cada árbol.
                                bootstrap=True, # usar subconjuntos con repetición de la base de datos para construír cada árbol.
                                random_state=10) # semilla aleatoria para reproducibilidad

# entrenamos el modelo
modelo_especifico = modelo_abstracto.fit(X=X_train, y=y_train)

In [ ]:
# evaluación del modelo
# predecimos para el conjunto de prueba
y_pred = modelo_especifico.predict(X_test)
 
# matriz de confusión
matriz_confusion = confusion_matrix(y_true=y_test, y_pred=y_pred)

# heatmap de la matriz de confusión
plt.figure(figsize=(8, 8))
sns.heatmap(matriz_confusion,
            annot=True, # anotar valor en cada cuadro
            fmt='d', # formato del valor que se mostrará en cada cuadro. 'd' hace referencia a 'sin decimales'
            cmap='Blues', # le estamos diciendo que use colores azules
            xticklabels=label_encoder.classes_, # leyenda del eje x
            yticklabels=label_encoder.classes_) # leyenda del eje y
plt.xlabel('Predicción') # título eje x
plt.ylabel('Condición Real') # título eje y
plt.title('Matriz de Confusión - Diagnóstico ECG') # título del gráfico
plt.show()

# tabla de resultados
# precision = True Positive / (True Positive + False Positive) = De los que predijiste con padecimiento X, cuántos si tienen el padecimiento.
# recall = True Positive / (True Positive + False Negative) = De los que tienen padecimiento X, cuántos predijiste correctamente.
# f1-score = Media armónica de precisión y recall = Penaliza valores bajos, por lo que ambos valores deben ser buenos para un valor alto.
# support = tamaño de muestra
# acurracy = precisión global = (Global True Positives + Global True Negatives) / number of events
# macro average = promedio de la columna
# weighted average = promedio de la columna ponderado por su tamaño de muestra
print("")
print("                 " + "\033[1m" + 'Reporte de clasificación' + "\033[1m")
print(classification_report(y_true=y_test, y_pred=y_pred, target_names=label_encoder.classes_))